In [ ]:
#!pip install langchain openai sentence-transformers
!pip install langchain sentence-transformers pandas nltk



In [25]:
#!pip install semchunk
!pip install git+https://github.com/isaacus-dev/semchunk.git


  Cloning https://github.com/isaacus-dev/semchunk.git to c:\users\mkolla1\appdata\local\temp\pip-req-build-cg4vavib
  Resolved https://github.com/isaacus-dev/semchunk.git to commit 9945642bca366925faf5bd969c2cf1bab1661725
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/isaacus-dev/semchunk.git 'C:\Users\mkolla1\AppData\Local\Temp\pip-req-build-cg4vavib'

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: C:\Users\mkolla1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## SemChunking

In [ ]:
import pandas as pd
import semchunk
import tiktoken
from semchunk import chunkerify
from sentence_transformers import SentenceTransformer
# Load a pre-trained embedding model (no API key required)
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Initialize Semantic Chunker
chunker = semchunk.chunkerify('gpt-4',chunk_size =50)  # Adjust threshold as needed
#chunker = semchunk.chunkerify(tiktoken.encoding_for_model('gpt-4'), max_token_chars = None,chunk_size =50)  # Adjust threshold as needed
# Process and chunk the law text
text= """ Author 1
Name: Manish Kolla
Major: Computer Science

Abstract:
CareerWide is an AI-powered recruitment platform designed to revolutionize the hiring process for both students and recruiters. By leveraging advanced AI, CareerWide streamlines recruitment through personalized, merit-based candidate matching and removes biases related to ethnicity, gender, and other personal characteristics. For students, the platform offers features like automated resume enhancement, personalized skill development calendars, and AI-driven insights to improve career prospects. Students can also express their individuality by selecting avatars, gaining visibility in a unique, inclusive way. For recruiters, CareerWide provides advanced filtering tools that focus on skills, GPA, and other relevant criteria, simplifying the search for the best candidates.      Author 2
Name: Nishchay Patel
Major: Computer Science. The platform also supports virtual career fairs, minimizing travel and promoting sustainability. With its focus on equity, efficiency, and innovation, CareerWide is transforming recruitment into a fairer, more sustainable, and inclusive process for all.
Faculty Sponsor: 
Name: Dr. Parag Tamhankar
Department: Computer Science
Email: ptamhankar@gsu.edu

     """
chunks = chunker(text, overlap= None)
for x in chunks:
  print(x)
  print('-------------------')
  print('-------------------')

In [14]:
file_path = "Title18_Pages_converted.csv"  # Change based on your file format
df = pd.read_csv(file_path)  
length=[]
for x in df['Text']:
    length.append(len(x))
print(sum(length)/len(length))

3647.8250311332504


In [ ]:
import pandas as pd
import semchunk
from semchunk import chunkerify
from sentence_transformers import SentenceTransformer

# Load the law text from a CSV file
file_path = "Title18_Pages_converted.csv"  # Change based on your file format
df = pd.read_csv(file_path)  

# Assume each page of the law text is in a column named 'content'
documents = df["Text"].tolist()

# Load a pre-trained embedding model (no API key required)
#embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Initialize Semantic Chunker
chunker = semchunk.chunkerify('gpt-4', max_token_chars = None,chunk_size =1000)  # Adjust threshold as needed

# Process and chunk the law text
chunks = chunker(documents)

# Convert chunked text into a DataFrame
chunked_data = []
for x in chunks:
    for y in x: 
        chunked_data.append([y])

chunked_df = pd.DataFrame(chunked_data, columns=["chunk"])

In [46]:
print(chunked_df.shape)

(2176, 1)


In [53]:
import random
num=random.randint(0, chunked_df.shape[0])
print(len(chunked_df['chunk'][num]))
print(type(chunked_df['chunk'][num]))
print(chunked_df['chunk'][num])

3344
<class 'str'>
Pub. L. 112–154, title VI, §601(a), Aug. 6, 2012, 126 Stat. 1195, provided that:
"(1) 
.—The purpose of this section [amending this section and section 2413 of Title 38, Veterans'
PURPOSE
Benefits] is to provide necessary and proper support for the recruitment and retention of the Armed Forces and
militia employed in the service of the United States by protecting the dignity of the service of the members of
such Forces and militia, and by protecting the privacy of their immediate family members and other attendees
during funeral services for such members.
"(2) 
.—Congress finds that this section is a necessary and proper
CONSTITUTIONAL AUTHORITY
exercise of its powers under the Constitution, article I, section 8, paragraphs 1, 12, 13, 14, 16, and 18, to
provide for the common defense, raise and support armies, provide and maintain a navy, make rules for the
government and regulation of the land and naval forces, and provide for organizing and governing such part of
t

In [54]:
# Save the results into a new CSV file
chunked_df.to_csv("chunked_title_18semchunk.csv", index=False)
print("✅ Semantic chunking completed! Saved to 'chunked_title_18semchunk.csv'.")

✅ Semantic chunking completed! Saved to 'chunked_title_18semchunk.csv'.


## RecursiveCharacterTextSplitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd

# Load the Title 18 law text from CSV
file_path = "Title18_Pages_converted.csv"  # Adjust for JSON/Excel if needed
df = pd.read_csv(file_path)  

# Assume each page of the law text is in a column named 'content'
documents = df["Text"].tolist()

# Use Hugging Face embeddings (local model, no API key needed)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize RecursiveCharacterTextSplitter for semantic-aware chunking
chunker = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Process and chunk the law text
chunks = []
for doc in documents:
    chunks.extend(chunker.split_text(doc))

# Convert chunked text into a DataFrame
chunked_df = pd.DataFrame({"chunk": chunks})

# Save the results into a new CSV file
chunked_df.to_csv("chunked_title_18RCTS.csv", index=False)

print("✅ Semantic chunking completed! Saved to 'chunked_title_18.csv'.")




print(chunked_df.shape)
print(chunked_df.head())


C:\Users\mkolla1\AppData\Local\Temp\ipykernel_33704\1983244083.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ Semantic chunking completed! Saved to 'chunked_title_18.csv'.
(7706, 1)
                                               chunk
0  6001\nImmunity of Witnesses\nV.\n5001\nCorrect...
1  37\n756, 3058\n38\nT. 22 §465\n39\n5, 3241\n51...
2  79\n1003\n80\n287, 1001\n81\n289\n82\n641, 136...
3  123\n912\n124\n211\n125\n543\n126\n541\n127\n1...
4  199\n205\n200\n204\n201\n1913\n202\n216\n203\n...


In [21]:
import random
print(chunked_df['chunk'][random.randint(0, chunked_df.shape[0])])

(p) Nothing in this section or sections 2248, 2259, 2264, 2327, 3663, and 3663A and arising out of
the application of such sections, shall be construed to create a cause of action not otherwise
authorized in favor of any person against the United States or any officer or employee of the United
States.
(Added Pub. L. 97–291, §5(a), Oct. 12, 1982, 96 Stat. 1255, §3580; renumbered §3664, Pub. L.
98–473, title II, §212(a)(1), Oct. 12, 1984, 98 Stat. 1987; amended Pub. L. 101–647, title XXXV,
§3596, Nov. 29, 1990, 104 Stat. 4931; Pub. L. 104–132, title II, §206(a), Apr. 24, 1996, 110 Stat.
1232; Pub. L. 107–273, div. B, title IV, §4002(e)(1), Nov. 2, 2002, 116 Stat. 1810.)
EDITORIAL NOTES
REFERENCES IN TEXT
The Federal Rules of Criminal Procedure, referred to in subsecs. (c) and (o)(1)(A), are set out in the
Appendix to this title.
AMENDMENTS
2002—Subsec. (o)(1)(C). Pub. L. 107–273 substituted "subsection (d)(5)" for "section 3664(d)(3)".


In [ ]:

# Export chunked text to a new CSV file
chunked_df.to_csv("chunked_title_18.csv", index=False)

print("Chunking completed! Saved to 'chunked_title_18.csv'.")

## Other Prototype (https://hasanaboulhasan.medium.com/the-best-text-chunking-method-f5faeb243d80#:~:text=The%20main%20idea%20behind%20semantic,cosine%20similarity%20between%20these%20chunks.)

In [1]:
import re
import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def chunk_text(text):
    # Split the input text into individual sentences.
    single_sentences_list = _split_sentences(text)
    # Combine adjacent sentences to form a context window around each sentence.
    combined_sentences = _combine_sentences(single_sentences_list)
    
    # Convert the combined sentences into vector representations using a neural network model.
    embeddings = convert_to_vector(combined_sentences)
    
    # Calculate the cosine distances between consecutive combined sentence embeddings to measure similarity.
    distances = _calculate_cosine_distances(embeddings)
    
    # Determine the threshold distance for identifying breakpoints based on the 80th percentile of all distances.
    breakpoint_percentile_threshold = 80
    breakpoint_distance_threshold = np.percentile(distances, breakpoint_percentile_threshold)
    # Find all indices where the distance exceeds the calculated threshold, indicating a potential chunk breakpoint.
    indices_above_thresh = [i for i, distance in enumerate(distances) if distance > breakpoint_distance_threshold]
    # Initialize the list of chunks and a variable to track the start of the next chunk.
    chunks = []
    start_index = 0
    # Loop through the identified breakpoints and create chunks accordingly.
    for index in indices_above_thresh:
        chunk = ' '.join(single_sentences_list[start_index:index+1])
        chunks.append(chunk)
        start_index = index + 1
    
    # If there are any sentences left after the last breakpoint, add them as the final chunk.
    if start_index < len(single_sentences_list):
        chunk = ' '.join(single_sentences_list[start_index:])
        chunks.append(chunk)
    
    # Return the list of text chunks.
    return chunks

def _split_sentences(text):
    # Use regular expressions to split the text into sentences based on punctuation followed by whitespace.
    sentences = re.split(r'(?<=[.?!])\s+', text)
    return sentences

def _combine_sentences(sentences):
    # Create a buffer by combining each sentence with its previous and next sentence to provide a wider context.
    combined_sentences = []
    for i in range(len(sentences)):
        combined_sentence = sentences[i]
        if i > 0:
            combined_sentence = sentences[i-1] + ' ' + combined_sentence
        if i < len(sentences) - 1:
            combined_sentence += ' ' + sentences[i+1]
        combined_sentences.append(combined_sentence)
    return combined_sentences
def convert_to_vector(texts):

    # Try to generate embeddings for a list of texts using a pre-trained model and handle any exceptions.
    try:
        response = openai.embeddings.create(
            input=texts,
            model="text-embedding-3-small"
        )
        embeddings = np.array([item.embedding for item in response.data])
        return embeddings
    except Exception as e:
        print("An error occurred:", e)
        return np.array([])  # Return an empty array in case of an error

def _calculate_cosine_distances(embeddings):
    # Calculate the cosine distance (1 - cosine similarity) between consecutive embeddings.
    distances = []
    for i in range(len(embeddings) - 1):
        similarity = cosine_similarity([embeddings[i]], [embeddings[i + 1]])[0][0]
        distance = 1 - similarity
        distances.append(distance)
    return distances

# Main Section
text= """ Author 1
Name: Manish Kolla
Major: Computer Science

Abstract:
CareerWide is an AI-powered recruitment platform designed to revolutionize the hiring process for both students and recruiters. By leveraging advanced AI, CareerWide streamlines recruitment through personalized, merit-based candidate matching and removes biases related to ethnicity, gender, and other personal characteristics. For students, the platform offers features like automated resume enhancement, personalized skill development calendars, and AI-driven insights to improve career prospects. Students can also express their individuality by selecting avatars, gaining visibility in a unique, inclusive way. For recruiters, CareerWide provides advanced filtering tools that focus on skills, GPA, and other relevant criteria, simplifying the search for the best candidates.      Author 2
Name: Nishchay Patel
Major: Computer Science. The platform also supports virtual career fairs, minimizing travel and promoting sustainability. With its focus on equity, efficiency, and innovation, CareerWide is transforming recruitment into a fairer, more sustainable, and inclusive process for all.
Faculty Sponsor: 
Name: Dr. Parag Tamhankar
Department: Computer Science
Email: ptamhankar@gsu.edu

     """
chunks = chunk_text(text)
print("Chunks:", chunks)

An error occurred: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


IndexError: index -1 is out of bounds for axis 0 with size 0